# *Implementación del modelo PointNet para segmentación de nubes de puntos*

### *Importaciones*

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import numpy as np
from pathlib import Path
import os
import tkinter as tk

### *Carga del Rutas*

In [12]:
# Crear una ventana oculta de Tkinter
from tkinter import filedialog

# Create a hidden Tkinter window
root = tk.Tk()
root.withdraw()  # Hide the main window

# Initial path
initial_path = Path(r'C:\Users\fmartinez\Desktop\reco\datasets_slipt')

try:
    # Select the first path
    print("Select the path for 'x_train'")
    x_train_path = Path(filedialog.askdirectory(
        title="Select the folder for 'x_train'",
        initialdir=initial_path
    ))

    # Select the second path
    print("Select the path for 'y_train'")
    y_train_path = Path(filedialog.askdirectory(
        title="Select the folder for 'y_train'",
        initialdir=initial_path
    ))

    # Select the third path
    print("Select the path for 'x_val'")
    x_val_path = Path(filedialog.askdirectory(
        title="Select the folder for 'x_val'",
        initialdir=initial_path
    ))

    # Select the fourth path
    print("Select the path for 'y_val'")
    y_val_path = Path(filedialog.askdirectory(
        title="Select the folder for 'y_val'",
        initialdir=initial_path
    ))

    # Show the selected paths
    print(f"Path selected for x_train: {x_train_path}")
    print(f"Path selected for y_train: {y_train_path}")
    print(f"Path selected for x_val: {x_val_path}")
    print(f"Path selected for y_val: {y_val_path}")

except Exception as e:
    print(f"Error selecting paths: {e}")

finally:
    root.destroy()  # Ensure the Tkinter window is closed

Select the path for 'x_train'
Select the path for 'y_train'
Select the path for 'x_val'
Select the path for 'y_val'
Path selected for x_train: C:\Users\fmartinez\Desktop\reco\datasets_slipt\goose_ex_val\x_train
Path selected for y_train: C:\Users\fmartinez\Desktop\reco\datasets_slipt\goose_ex_val\y_train
Path selected for x_val: C:\Users\fmartinez\Desktop\reco\datasets_slipt\goose_ex_val\x_val
Path selected for y_val: C:\Users\fmartinez\Desktop\reco\datasets_slipt\goose_ex_val\y_val


### *Carga de Datos*

In [13]:
import pandas as pd
from pathlib import Path

def load_data(x_path, y_path):
    X_data, Y_data = [], []

    for x_file in Path(x_path).iterdir():
        if x_file.is_file() and x_file.suffix == '.csv':
            y_file = Path(y_path) / x_file.name.replace('x_', 'y_')

            # Load x and y
            x_df = pd.read_csv(x_file, header=None, skiprows=1)  # Omitir la primera fila si es encabezado
            y_df = pd.read_csv(y_file, header=None, skiprows=1)  # Omitir la primera fila si es encabezado

            try:
                # Convert to float and select x, y, z features
                x_features = x_df.iloc[:, [0, 1, 2]].astype(float).values
                labels = y_df.iloc[:, 0].values

                X_data.append(x_features)
                Y_data.append(labels)
            except ValueError:
                print(f"Error en el archivo: {x_file}, revisa que los datos sean numéricos.")

    return X_data, Y_data

# Load training and validation data
x_train, y_train = load_data(x_train_path, y_train_path)
x_val, y_val = load_data(x_val_path, y_val_path)


In [4]:
x_train[0]

array([[ 0.01638596, -0.00336316,  0.0778692 ],
       [ 0.01643232, -0.00350216,  0.07763668],
       [ 0.01649578, -0.00322229,  0.07769629],
       ...,
       [-0.03970338, -0.00906141,  0.04173246],
       [-0.03696398, -0.00789607,  0.04225726],
       [-0.04472937, -0.00298752,  0.04224357]])

In [14]:
num_classes = 64

# Calculate class weights
def calculate_class_weights(y_data):
    all_labels = np.concatenate(y_data)
    class_counts = np.bincount(all_labels, minlength=num_classes)
    total_samples = len(all_labels)
    class_weights = {i: total_samples / (num_classes * count) for i, count in enumerate(class_counts) if count > 0}
    return class_weights

class_weights = calculate_class_weights(y_train)
print(f"Calculated class weights: {class_weights}")

Calculated class weights: {0: 27.558065379208998, 1: 1162.5641004043127, 3: 1.838876669942422, 4: 0.8149673468905737, 5: 16.61797689225375, 6: 55.68540200761733, 8: 0.08017257215249826, 10: 6296.5150547445255, 11: 215.9796100400601, 12: 13.399129568648162, 13: 758.0163115114235, 14: 6.857967328913057, 16: 1.2928452680343152, 17: 1.0124749264666184, 20: 287540.8541666667, 21: 87.08960752145381, 22: 40.966071258963765, 23: 0.25610316611356243, 24: 1.051923572910201, 26: 98.61924802789528, 27: 0.557797425443588, 28: 2.218468310628875, 29: 0.32896550662261254, 31: 0.04672188738597281, 34: 9.045009567998322, 37: 52.83087717417932, 38: 2.422158159196716, 39: 1104.5103233034572, 40: 2.122124929272553, 41: 3.647252212351964, 42: 4.186207919422701, 45: 31.47108947464429, 46: 506.23389818075117, 47: 23.14025866462793, 50: 0.07965990767966621, 51: 0.6966341502513179, 54: 9.634903692575756, 55: 619.7001167385057, 57: 4.189623655261177, 58: 120.30998082287309, 59: 35.78752748506472, 60: 11.51622138

## Implementacion de PointNet

En primer lugar se define el bloque T-Net para el ajuste espacial mediante una matriz de transformación que aprende. De esta manera el modelo es robusto a las posibles rotaciones que los objetos puedan tener en el espacio. Se utilizan capas convolucionales 1D que funcionan como MPL pero aunemtan la eficiencia computacional del modelo

In [1]:
# -----------------------------
# DEFINICIÓN DEL MODELO POINTNET
# -----------------------------
def build_pointnet(num_classes, input_dim=3):
    """
    Modelo PointNet para segmentación punto a punto.
    """
    inputs = tf.keras.Input(shape=(None, input_dim))  # Entrada: N puntos con D características

    # Capa convolucional inicial (clasificación)
    x = layers.Conv1D(64, 1, activation='relu')(inputs) # 64 Filtros; Kernel_dim = 1 (punto por punto)
    x = layers.Conv1D(128, 1, activation='relu')(x)
    x = layers.Conv1D(1024, 1, activation='relu')(x)

  # Agregación de características globales
    global_features = layers.GlobalMaxPooling1D()(x)  # Max pooling global
    global_features = tf.expand_dims(global_features, axis=1)  # Expandir dimensión para repetir
    num_points = tf.shape(inputs)[1]  # Obtener número dinámico de puntos
    global_features = tf.tile(global_features, [1, num_points, 1])  # Repetir dinámicamente  

    # Concatenar características globales y locales
    x = layers.concatenate([x, global_features])

    # Capa convolucional final para segmentación
    x = layers.Conv1D(512, 1, activation='relu')(x)
    x = layers.Conv1D(256, 1, activation='relu')(x)
    outputs = layers.Conv1D(num_classes, 1, activation='softmax')(x)  # Segmentacion por punto

    return tf.keras.Model(inputs, outputs)

In [16]:
# Instantiate the model
num_classes = 64
model = pointnet_segmentation(num_points=None, num_classes=num_classes)
model.summary()

# Define custom metrics for semantic segmentation
class MeanIoU(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name="mean_iou", **kwargs):
        super(MeanIoU, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.confusion_matrix = self.add_weight(name="conf_matrix", shape=(num_classes, num_classes), initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.reshape(y_true, [-1])
        y_pred = tf.argmax(y_pred, axis=-1)
        y_pred = tf.reshape(y_pred, [-1])

        current_conf_matrix = tf.math.confusion_matrix(y_true, y_pred, num_classes=self.num_classes, dtype=tf.float32)
        self.confusion_matrix.assign_add(current_conf_matrix)

    def result(self):
        cm = self.confusion_matrix
        intersection = tf.linalg.diag_part(cm)
        ground_truth_set = tf.reduce_sum(cm, axis=1)
        predicted_set = tf.reduce_sum(cm, axis=0)
        union = ground_truth_set + predicted_set - intersection
        iou = intersection / tf.maximum(union, tf.keras.backend.epsilon())
        return tf.reduce_mean(iou)

    def reset_states(self):
        self.confusion_matrix.assign(tf.zeros_like(self.confusion_matrix))

# Compile the model with custom metrics
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', MeanIoU(num_classes=num_classes), 'mae', tf.keras.metrics.Precision(name='precision')])

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None, 3)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, None, 64)  │        256 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, None, 128) │      8,320 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, None,      │    132,096 │ conv1d_1[0][0]    │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 1024)      │          0 │ conv1d_2[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │    524,800 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │    131,328 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 9)         │      2,313 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 3, 3)      │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, None, 3)   │          0 │ input_layer[0][0… │
│                     │                   │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, None, 64)  │        256 │ dot[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, None, 64)  │      4,160 │ conv1d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, None, 64)  │      4,160 │ conv1d_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, None, 128) │      8,320 │ conv1d_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, None,      │    132,096 │ conv1d_6[0][0]    │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 1024)      │          0 │ conv1d_7[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 512)       │    524,800 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │    131,328 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 4096)      │  1,052,672 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 64, 64)    │          0 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_1 (Dot)         │ (None, None, 64)  │          0 │ conv1d_4[0][0],   │
│                     │                   │            │ reshape_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 3,531,529 (13.47 MB)

 Trainable params: 3,531,529 (13.47 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Prepare data for training (batch generator)
def data_generator_with_weights(X, Y, batch_size, class_weights):
    while True:
        indices = np.random.choice(len(X), batch_size) 
        batch_X, batch_Y, batch_weights = [], [], []

        for idx in indices:
            x_sample = X[idx]
            y_sample = Y[idx]

            batch_X.append(x_sample)
            batch_Y.append(y_sample)

            # Assign weights to each label
            weights = np.array([class_weights[label] for label in y_sample])
            batch_weights.append(weights)

        batch_X = tf.ragged.constant(batch_X, dtype=tf.float32).to_tensor()
        batch_Y = tf.ragged.constant(batch_Y, dtype=tf.int32).to_tensor()
        batch_weights = tf.ragged.constant(batch_weights, dtype=tf.float32).to_tensor()

        yield batch_X, batch_Y, batch_weights

In [18]:
# Train the model
batch_size = 16
epochs = 10

train_generator = data_generator_with_weights(x_train, y_train, batch_size, class_weights)
val_generator = data_generator_with_weights(x_val, y_val, batch_size, class_weights)

# Define a callback to show training progress dynamically
class TrainingProgressCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(f"Epoch {epoch + 1}:")
        for key, value in logs.items():
            print(f"  {key}: {value:.4f}")

# Train the model with the callback
history = model.fit(
    train_generator,
    validation_data=val_generator,
    steps_per_epoch=len(x_train) // batch_size,
    validation_steps=len(x_val) // batch_size,
    epochs=epochs,
    callbacks=[TrainingProgressCallback()]
)

# Plot training progress
import matplotlib.pyplot as plt

def plot_training_progress(history):
    plt.figure(figsize=(12, 4))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_training_progress(history)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\fmartinez\AppData\Local\anaconda3\envs\advancedUAV\lib\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\fmartinez\AppData\Local\Temp\ipykernel_12552\1365324335.py", line 17, in <module>
    history = model.fit(
  File "c:\Users\fmartinez\AppData\Local\anaconda3\envs\advancedUAV\lib\site-packages\keras\src\utils\traceback_utils.py", line 117, in error_handler
    else:
  File "c:\Users\fmartinez\AppData\Local\anaconda3\envs\advancedUAV\lib\site-packages\keras\src\backend\tensorflow\trainer.py", line 332, in fit
  File "c:\Users\fmartinez\AppData\Local\anaconda3\envs\advancedUAV\lib\site-packages\keras\src\backend\tensorflow\trainer.py", line 720, in __init__
  File "c:\Users\fmartinez\AppData\Local\anaconda3\envs\advancedUAV\lib\site-packages\keras\src\trainers\epoch_iterator.py", line 65, in __init__
  File "c:\Users\fmartinez\AppData\Local\a

In [7]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Devices:", tf.config.list_physical_devices())



Num GPUs Available:  0
Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
